In [20]:
import pandas as pd
import numpy as np

# Specify the path to the pickle file
data_file_path = '../Dataset/Dementia_paper_dataset_data.pkl'
info_file_path = '../Dataset/Dementia_paper_dataset_info.pkl'

# Open the pickle file in read mode
with open(data_file_path, 'rb') as file:
    # Load the data from the pickle file
    data = pd.read_pickle(file)

# Open the pickle file in read mode
with open(info_file_path, 'rb') as file:
    # Load the data from the pickle file
    info = pd.read_pickle(file)

# Display the first few rows of the data
data['train'].head()

,ID,Task,Coh_delta_T5-T3,Coh_delta_T5-F7,Coh_delta_T5-O1,Coh_delta_T5-Cp3,Coh_delta_T5-Fc3,Coh_delta_T5-Fp1,Coh_delta_T5-Fcz,Coh_delta_T5-Cpz,...,Corr_Cp4-O2,Corr_Cp4-F8,Corr_Cp4-T4,Corr_Cp4-T6,Corr_O2-F8,Corr_O2-T4,Corr_O2-T6,Corr_F8-T4,Corr_F8-T6,Corr_T4-T6
0,NTUH_0004,Rest,0.739369,0.701256,0.759699,0.741979,0.482286,0.165846,0.110406,0.291415,...,0.765194,0.567678,0.708985,0.803361,0.404576,0.661505,0.919683,0.717551,0.520095,0.783182
1,CGMHKL_0002,Rest,0.814006,0.304687,0.923202,0.659374,0.235388,0.263912,0.233530,0.494247,...,0.594284,0.311152,0.518404,0.688676,0.192620,0.477090,0.762633,0.449235,0.383965,0.661592
2,NTUH_0011,Rest,0.756742,0.273296,0.838798,0.849737,0.255621,0.145847,0.172226,0.420560,...,0.648830,0.432738,0.592082,0.750562,0.244982,0.558273,0.891249,0.643338,0.400314,0.701446
3,CGMHKL_0005,Rest,0.755170,0.332022,0.721507,0.514757,0.138277,0.259687,0.193953,0.307590,...,0.614496,0.296134,0.239646,0.736570,0.284022,0.345613,0.818268,0.302792,0.371725,0.428486
4,CGMHKL_0004,Rest,0.651956,0.284282,0.611058,0.573420,0.240884,0.169931,0.155174,0.229636,...,0.347948,0.311138,0.429988,0.502850,0.242461,0.377395,0.727081,0.626247,0.420851,0.575280


In [32]:
# Dataframe to numpy array in each dictionary
data_ndarrays = {}
for key, value in data.items():
    # Detect if the key is 'Info' and skip it
    if key != 'Info':
        # Drop the 'ID' and 'Task' columns if they exist
        if 'ID' in data[key].columns:
            data[key].drop('ID', axis=1, inplace=True)
        if 'Task' in data[key].columns:
            data[key].drop('Task', axis=1, inplace=True)  
        data_ndarrays[key] = value.values
    else:
        data_ndarrays[key] = value

label_ndarrays = {}
for key, value in info.items():
    if key != 'Info':
        label_ndarrays[key] = value['Label'].values
    else:
        label_ndarrays[key] = value

In [34]:
from feature_related import feature_selection as f_selection
f_selection = f_selection()
fisher_scores = f_selection.fisher_score(data_ndarrays['train'], label_ndarrays['train'])
fisher_idx = np.argsort(fisher_scores)[::-1] # sort in descending order

In [54]:
# classifying the data using LDA and LOO-CV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline
from sklearn.model_selection import LeaveOneOut
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

lda = LinearDiscriminantAnalysis()
qda = QuadraticDiscriminantAnalysis()
# Create the pipeline
pipeline = Pipeline([
    ('lda', lda)
])

looCV = LeaveOneOut()
accCV = 0
# Loop through the indices the looCV.split() method returns
data_CV = data_ndarrays['train'][:, fisher_idx[:10]]
for i, (train_idx, valid_idx) in enumerate(looCV.split(data_ndarrays['train'])):
    # Train the model
    pipeline.fit(data_CV[train_idx], label_ndarrays['train'][train_idx])
    # Test the model
    score = pipeline.score(data_CV[valid_idx], label_ndarrays['train'][valid_idx])
    accCV += score
    print('Prediction:', score, end=' ')
    print('True label:', label_ndarrays['train'][valid_idx])
print('Accuracy:', accCV / (i + 1))

Prediction: 0.0 True label: [1]
Prediction: 1.0 True label: [0]
Prediction: 1.0 True label: [0]
Prediction: 0.0 True label: [0]
Prediction: 0.0 True label: [0]
Prediction: 1.0 True label: [0]
Prediction: 0.0 True label: [1]
Prediction: 1.0 True label: [0]
Prediction: 0.0 True label: [0]
Prediction: 1.0 True label: [1]
Prediction: 1.0 True label: [0]
Prediction: 1.0 True label: [1]
Prediction: 0.0 True label: [1]
Prediction: 1.0 True label: [0]
Prediction: 1.0 True label: [1]
Prediction: 1.0 True label: [0]
Prediction: 0.0 True label: [0]
Prediction: 0.0 True label: [1]
Prediction: 1.0 True label: [0]
Prediction: 1.0 True label: [0]
Prediction: 0.0 True label: [1]
Prediction: 1.0 True label: [0]
Prediction: 1.0 True label: [0]
Prediction: 1.0 True label: [0]
Prediction: 1.0 True label: [1]
Prediction: 1.0 True label: [0]
Prediction: 1.0 True label: [1]
Prediction: 1.0 True label: [1]
Prediction: 1.0 True label: [1]
Prediction: 1.0 True label: [1]
Prediction: 1.0 True label: [2]
Predicti

In [46]:
fisher_idx[:10]

array([10857, 10596, 10883,   707,  8845,   733, 12416, 10610, 12413,
         446], dtype=int64)